# 🌡️ Implement Temperature Sampling for LLM Decoding

### Problem Statement

Implement **Temperature Sampling**, the fundamental technique for controlling randomness in language model text generation. Temperature scaling modifies the probability distribution over the vocabulary, allowing you to trade off between creative/diverse outputs and deterministic/focused outputs.

This is one of the most commonly asked topics in AI engineer interviews because it's essential for understanding how ChatGPT, Claude, and other LLMs generate text.

---

### Background: Why Temperature?

**The Problem with Greedy Decoding**

Language models output **logits** (raw scores) for each token in the vocabulary. The naive approach is to always pick the highest-scoring token (greedy decoding), but this leads to:
- Repetitive, boring text
- Getting stuck in loops
- Missing creative or contextually appropriate alternatives

**The Solution: Sampling with Temperature**

Instead of always picking the argmax, we:
1. Convert logits to probabilities using softmax
2. **Scale the logits by temperature** before softmax
3. Sample from the resulting distribution

**Temperature Effects:**

| Temperature | Effect | Use Case |
|-------------|--------|----------|
| T → 0 | Approaches greedy (argmax) | Factual Q&A, code generation |
| T = 1.0 | Original distribution | Balanced generation |
| T > 1.0 | Flatter distribution (more random) | Creative writing, brainstorming |

**Real-World Defaults:**
- ChatGPT: T ≈ 0.7 for chat, T ≈ 0.2 for code
- Claude: T ≈ 1.0 with other sampling methods
- Code generation: T ≈ 0.0-0.2 for deterministic output

---

### Mathematical Formulation

**Standard Softmax:**
$$p_i = \frac{e^{z_i}}{\sum_j e^{z_j}}$$

**Softmax with Temperature:**
$$p_i = \frac{e^{z_i / T}}{\sum_j e^{z_j / T}}$$

Where:
- $z_i$ = logit for token $i$
- $T$ = temperature parameter
- $p_i$ = probability of token $i$

**Why Division by T Works:**

- **T < 1**: Dividing by a small number **amplifies** differences between logits → sharper distribution
- **T = 1**: No change → original distribution
- **T > 1**: Dividing by a large number **reduces** differences between logits → flatter distribution

**Limit Behavior:**
- As T → 0: Distribution becomes one-hot (argmax)
- As T → ∞: Distribution becomes uniform

---

### Learning Objectives

By completing this exercise, you will:

1. ✅ Understand how temperature affects probability distributions
2. ✅ Implement temperature-scaled softmax from scratch
3. ✅ Visualize the effect of different temperatures
4. ✅ Sample tokens from the modified distribution
5. ✅ Understand the trade-off between creativity and determinism

---

### Requirements

Implement the following functions:

1. **`softmax_with_temperature(logits, temperature)`**
   - Apply temperature scaling to logits
   - Return probability distribution

2. **`sample_with_temperature(logits, temperature)`**
   - Get probabilities using temperature
   - Sample a token index from the distribution

3. **`generate_with_temperature(model_fn, prompt_tokens, max_tokens, temperature)`**
   - Autoregressive generation loop
   - Use temperature sampling at each step

---

### Hints

<details>
  <summary>💡 Hint 1: Temperature Scaling</summary>
  
  Simply divide the logits by temperature before applying softmax:
  ```python
  scaled_logits = logits / temperature
  probs = F.softmax(scaled_logits, dim=-1)
  ```
  
  Handle the edge case where temperature is very small (use a minimum like 1e-8).
</details>

<details>
  <summary>💡 Hint 2: Sampling from Distribution</summary>
  
  Use `torch.multinomial` to sample from a probability distribution:
  ```python
  token_idx = torch.multinomial(probs, num_samples=1)
  ```
  
  This randomly selects an index with probability proportional to `probs`.
</details>

<details>
  <summary>💡 Hint 3: Numerical Stability</summary>
  
  For numerical stability with very low temperatures, subtract the max logit before exponentiating:
  ```python
  scaled = logits / temperature
  scaled = scaled - scaled.max(dim=-1, keepdim=True).values
  ```
  
  This prevents overflow in `exp()` while preserving the softmax result.
</details>

<details>
  <summary>💡 Hint 4: Generation Loop</summary>
  
  The autoregressive generation pattern:
  ```python
  for _ in range(max_tokens):
      logits = model(tokens)[:, -1, :]  # Get logits for last position
      next_token = sample_with_temperature(logits, temperature)
      tokens = torch.cat([tokens, next_token], dim=-1)
  ```
</details>

---

### Implementation

In [ ]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def softmax_with_temperature(logits: torch.Tensor, temperature: float = 1.0) -> torch.Tensor:
    """
    Apply temperature-scaled softmax to logits.
    
    Args:
        logits: Raw model outputs, shape (batch_size, vocab_size) or (vocab_size,)
        temperature: Temperature parameter (default 1.0)
                    - T < 1.0: Sharper distribution (more deterministic)
                    - T = 1.0: Original distribution
                    - T > 1.0: Flatter distribution (more random)
    
    Returns:
        Probability distribution over vocabulary, same shape as input
    
    Example:
        >>> logits = torch.tensor([2.0, 1.0, 0.1])
        >>> softmax_with_temperature(logits, temperature=0.5)  # Sharper
        >>> softmax_with_temperature(logits, temperature=2.0)  # Flatter
    """
    # TODO: Implement temperature-scaled softmax
    # 1. Ensure temperature is positive (avoid division by zero)
    # 2. Scale logits by temperature
    # 3. Apply softmax
    
    pass

In [ ]:
def sample_with_temperature(logits: torch.Tensor, temperature: float = 1.0) -> torch.Tensor:
    """
    Sample a token index from logits using temperature scaling.
    
    Args:
        logits: Raw model outputs, shape (batch_size, vocab_size) or (vocab_size,)
        temperature: Temperature parameter for controlling randomness
    
    Returns:
        Sampled token index, shape (batch_size, 1) or (1,)
    
    Example:
        >>> logits = torch.tensor([2.0, 1.0, 0.1])
        >>> sample_with_temperature(logits, temperature=1.0)
        tensor([0])  # Likely to be 0 (highest logit)
    """
    # TODO: Implement temperature sampling
    # 1. Get temperature-scaled probabilities
    # 2. Handle 1D case (add batch dimension)
    # 3. Sample from the distribution using torch.multinomial
    
    pass

In [ ]:
def generate_with_temperature(
    model_fn,
    prompt_tokens: torch.Tensor,
    max_new_tokens: int,
    temperature: float = 1.0,
    eos_token_id: int = None
) -> torch.Tensor:
    """
    Generate tokens autoregressively using temperature sampling.
    
    Args:
        model_fn: Function that takes tokens and returns logits
                  Signature: model_fn(tokens) -> logits of shape (batch, seq_len, vocab_size)
        prompt_tokens: Initial tokens, shape (batch_size, seq_len)
        max_new_tokens: Maximum number of tokens to generate
        temperature: Temperature for sampling
        eos_token_id: Optional end-of-sequence token ID to stop generation
    
    Returns:
        Generated tokens including prompt, shape (batch_size, seq_len + generated)
    """
    # TODO: Implement autoregressive generation
    # 1. Clone prompt tokens
    # 2. Loop for max_new_tokens:
    #    a. Get model predictions
    #    b. Extract logits for last position
    #    c. Sample next token with temperature
    #    d. Append to sequence
    #    e. Check for EOS
    
    pass

### Visualization: Temperature Effects

Let's visualize how temperature affects the probability distribution.

In [ ]:
# Create sample logits (simulating model output for 10 tokens)
torch.manual_seed(42)
logits = torch.tensor([2.5, 2.0, 1.5, 1.0, 0.5, 0.0, -0.5, -1.0, -1.5, -2.0])
token_names = [f"token_{i}" for i in range(len(logits))]

# Test different temperatures
temperatures = [0.1, 0.5, 1.0, 2.0, 5.0]

fig, axes = plt.subplots(1, len(temperatures), figsize=(15, 3))
fig.suptitle('Effect of Temperature on Probability Distribution', fontsize=14)

for ax, temp in zip(axes, temperatures):
    probs = softmax_with_temperature(logits, temperature=temp)
    ax.bar(range(len(probs)), probs.numpy())
    ax.set_title(f'T = {temp}')
    ax.set_ylim(0, 1)
    ax.set_xlabel('Token')
    if temp == temperatures[0]:
        ax.set_ylabel('Probability')
    
    # Show entropy
    entropy = -(probs * torch.log(probs + 1e-10)).sum().item()
    ax.text(0.5, 0.95, f'H={entropy:.2f}', transform=ax.transAxes, 
            verticalalignment='top', fontsize=10)

plt.tight_layout()
plt.show()

print("\nObservations:")
print("- T=0.1: Almost deterministic (near-argmax)")
print("- T=0.5: Sharp but still some variation")
print("- T=1.0: Original distribution")
print("- T=2.0: Flatter, more random")
print("- T=5.0: Nearly uniform distribution")

### Testing

In [ ]:
def test_softmax_with_temperature():
    """Test that temperature scaling works correctly."""
    logits = torch.tensor([2.0, 1.0, 0.0])
    
    # Test 1: Temperature = 1.0 should equal standard softmax
    probs_t1 = softmax_with_temperature(logits, temperature=1.0)
    probs_std = F.softmax(logits, dim=-1)
    assert torch.allclose(probs_t1, probs_std, atol=1e-6), "T=1.0 should equal standard softmax"
    print("✓ T=1.0 equals standard softmax")
    
    # Test 2: Probabilities should sum to 1
    for temp in [0.1, 0.5, 1.0, 2.0, 5.0]:
        probs = softmax_with_temperature(logits, temperature=temp)
        assert torch.allclose(probs.sum(), torch.tensor(1.0), atol=1e-6), f"Probs should sum to 1 at T={temp}"
    print("✓ Probabilities sum to 1 for all temperatures")
    
    # Test 3: Lower temperature should make distribution sharper
    probs_low = softmax_with_temperature(logits, temperature=0.1)
    probs_high = softmax_with_temperature(logits, temperature=2.0)
    assert probs_low.max() > probs_high.max(), "Lower T should have higher max probability"
    print("✓ Lower temperature creates sharper distribution")
    
    # Test 4: Very low temperature should approach argmax
    probs_tiny = softmax_with_temperature(logits, temperature=0.01)
    assert probs_tiny.argmax() == logits.argmax(), "Very low T should select argmax"
    assert probs_tiny.max() > 0.99, "Very low T should have near-1.0 max probability"
    print("✓ Very low temperature approaches argmax")
    
    print("\n✓ All softmax_with_temperature tests passed!")


def test_sample_with_temperature():
    """Test sampling behavior."""
    torch.manual_seed(42)
    logits = torch.tensor([10.0, 0.0, 0.0, 0.0, 0.0])  # Strong preference for first token
    
    # Test 1: Low temperature should almost always select argmax
    samples_low_t = [sample_with_temperature(logits, temperature=0.01).item() for _ in range(100)]
    assert sum(s == 0 for s in samples_low_t) > 95, "Low T should select argmax most of the time"
    print("✓ Low temperature selects argmax consistently")
    
    # Test 2: High temperature should give more variety
    samples_high_t = [sample_with_temperature(logits, temperature=5.0).item() for _ in range(100)]
    unique_samples = len(set(samples_high_t))
    assert unique_samples > 1, "High T should produce variety"
    print(f"✓ High temperature produces {unique_samples} unique samples")
    
    # Test 3: Output shape should be correct
    batch_logits = torch.randn(4, 100)  # Batch of 4, vocab size 100
    samples = sample_with_temperature(batch_logits, temperature=1.0)
    assert samples.shape == (4, 1), f"Expected shape (4, 1), got {samples.shape}"
    print("✓ Batch sampling produces correct shape")
    
    print("\n✓ All sample_with_temperature tests passed!")


def test_generate_with_temperature():
    """Test generation loop."""
    # Create a simple mock model
    vocab_size = 10
    def mock_model(tokens):
        # Returns random logits for testing
        batch_size, seq_len = tokens.shape
        return torch.randn(batch_size, seq_len, vocab_size)
    
    # Test generation
    prompt = torch.tensor([[0, 1, 2]])  # Batch size 1, 3 prompt tokens
    generated = generate_with_temperature(mock_model, prompt, max_new_tokens=5, temperature=1.0)
    
    assert generated.shape == (1, 8), f"Expected shape (1, 8), got {generated.shape}"
    assert (generated[:, :3] == prompt).all(), "Prompt should be preserved"
    print("✓ Generation produces correct output shape")
    print("✓ Prompt tokens are preserved")
    
    print("\n✓ All generate_with_temperature tests passed!")


# Run all tests
test_softmax_with_temperature()
print()
test_sample_with_temperature()
print()
test_generate_with_temperature()

print("\n" + "="*60)
print("✓ All tests passed! Temperature sampling implementation is correct.")
print("="*60)

### Practical Example: Sampling Diversity

Let's see how temperature affects the diversity of generated samples.

In [ ]:
# Simulate vocabulary with meaningful tokens
vocab = ["the", "a", "quick", "brown", "fox", "jumped", "over", "lazy", "dog", "cat"]

# Simulated logits favoring certain tokens
logits = torch.tensor([3.0, 2.5, 1.0, 1.5, 0.5, 0.8, 0.3, 0.2, -0.5, -1.0])

print("Sampling 10 tokens at different temperatures:\n")

for temp in [0.1, 0.5, 1.0, 2.0]:
    samples = [vocab[sample_with_temperature(logits, temp).item()] for _ in range(10)]
    unique_count = len(set(samples))
    print(f"T={temp}: {samples}")
    print(f"        Unique tokens: {unique_count}/10\n")

## Summary

### Key Concepts

- **Temperature Scaling**: Divide logits by temperature before softmax to control distribution sharpness
- **Low Temperature (T < 1)**: More deterministic, picks high-probability tokens
- **High Temperature (T > 1)**: More random, flatter distribution, more diverse outputs
- **T = 1.0**: Original model distribution
- **Sampling**: Use `torch.multinomial` to sample from the probability distribution

### When to Use Different Temperatures

| Use Case | Temperature | Why |
|----------|-------------|-----|
| Code generation | 0.0-0.2 | Deterministic, correct syntax |
| Factual Q&A | 0.3-0.5 | Focused but not repetitive |
| General chat | 0.7-0.9 | Balanced creativity |
| Creative writing | 1.0-1.5 | More diverse outputs |
| Brainstorming | 1.5-2.0 | Maximum creativity |

---

## Interview Tips

Be ready to answer:

**Q: What is temperature in LLM sampling?**
- A: Temperature is a hyperparameter that scales logits before softmax, controlling the randomness of text generation. Lower temperatures make the model more deterministic (sharper distribution), higher temperatures make it more random (flatter distribution).

**Q: What happens when temperature approaches 0?**
- A: The distribution becomes increasingly peaked, approaching a one-hot vector at the argmax. This is equivalent to greedy decoding. Mathematically, dividing by a very small T amplifies differences between logits.

**Q: What happens when temperature is very high?**
- A: The distribution becomes increasingly uniform. Dividing by a large T compresses all logits toward zero, making the softmax output approach 1/vocab_size for all tokens.

**Q: Why do we divide logits by temperature instead of multiplying?**
- A: Division makes the intuition clearer: T<1 "sharpens" and T>1 "softens". Mathematically, `logits/T` with T<1 amplifies differences between logits, making high logits even more dominant.

**Q: What's the trade-off between low and high temperature?**
- A: Low temperature: more accurate/focused but potentially repetitive and boring. High temperature: more creative/diverse but may produce incoherent or off-topic text. The optimal value depends on the task.

**Q: How does temperature interact with other sampling methods like top-k and top-p?**
- A: Temperature is typically applied first (scaling logits), then top-k/top-p filtering is applied to the resulting probabilities. Common combination: temperature=0.7 with top_p=0.9. The order matters!

**Q: What temperature would you use for code generation vs creative writing?**
- A: Code generation: T≈0.0-0.2 for deterministic, syntactically correct output. Creative writing: T≈1.0-1.5 for more diverse, unexpected word choices. Always validate empirically for your specific use case.

---

## References

- [The Curious Case of Neural Text Degeneration (Holtzman et al., 2020)](https://arxiv.org/abs/1904.09751) - Introduces nucleus sampling, discusses temperature
- [Language Models are Few-Shot Learners (Brown et al., 2020)](https://arxiv.org/abs/2005.14165) - GPT-3 paper, discusses sampling strategies
- [How to Generate Text: Using Different Decoding Methods (HuggingFace)](https://huggingface.co/blog/how-to-generate) - Practical guide to generation
- [Temperature in Softmax (Wikipedia)](https://en.wikipedia.org/wiki/Softmax_function#Softmax_with_temperature) - Mathematical background